### 데이터 재구조화
- pivot, pd.pivot_table
- stack, unstack
- melt
- wide_to_long
- pd.crosstab

### pivot, pd.pivot_table

In [8]:
import numpy as np
import pandas as pd
data = pd.DataFrame({
    'cust_id': ['c1', 'c1', 'c1', 'c2', 'c2', 'c2', 'c3', 'c3', 'c3'],
    'prod_cd': ['p1', 'p2', 'p3', 'p1', 'p2', 'p3', 'p1', 'p2', 'p3'],
     'grade' : ['A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B'],
    'pch_amt': [30, 10, 0, 40, 15, 30, 0, 0, 10]})
print(data, "\n")

# 데이터 재구조화 : data.pivot(index, columns, values)
data_pivot = data.pivot(index   = 'cust_id', 
                        columns = 'prod_cd', 
                        values  = 'pch_amt')

print(data_pivot, '\n')

# 데이터 재구조화 : data.pivot(index, columns, values, aggfunc)
data_pivot_2 = pd.pivot_table(data, 
                              index   = 'cust_id', 
                              columns = 'prod_cd', 
                              values  = 'pch_amt')

print(data_pivot_2, '\n')

# data.pivot으로 하면 에러가 나서 안되고, pd.pivot_table(data)을 사용해야만 하는
# 경우가 몇 가지 있음. 따라서 왠만하면 pivot_table을 사용하는 것이 좋음
# (1) index가 2개 이상인 경우
data_pivot_multiIndex = pd.pivot_table(data, 
                              index   = ['cust_id', 'grade'], 
                              columns = 'prod_cd', 
                              values  = 'pch_amt')
print(data_pivot_multiIndex, '\n')

# (2) column이 2개 이상인 경우
data_pivot_multiCol = pd.pivot_table(data, 
                              index   = 'cust_id', 
                              columns = ['grade','prod_cd'], 
                              values  = 'pch_amt')
print(data_pivot_multiCol, '\n')

# data.pivot 함수는 중복값이 존재할 경우, error
# pd.pivot_table 함수는 중복시 aggfunc을 사용하여 문제 없이 해결 가능
data_pivot_sum = pd.pivot_table(data, 
                              index   = 'grade', 
                              columns = 'prod_cd', 
                              values  = 'pch_amt',
                              aggfunc = np.sum) # np.mean..
print(data_pivot_sum, '\n')

# pd.pivot_table은 margins = True 옵션을 지정해주면, 행과 열을 기준으로
# 합계(All, row sum, column sum)를 같이 제시해 줌
data_pivot_margins = pd.pivot_table(data, 
                              index   = 'grade', 
                              columns = 'prod_cd', 
                              values  = 'pch_amt',
                              aggfunc = np.sum,
                              margins = True) # np.mean..
print(data_pivot_margins, '\n')

  cust_id prod_cd grade  pch_amt
0      c1      p1     A       30
1      c1      p2     A       10
2      c1      p3     A        0
3      c2      p1     A       40
4      c2      p2     A       15
5      c2      p3     A       30
6      c3      p1     B        0
7      c3      p2     B        0
8      c3      p3     B       10 

prod_cd  p1  p2  p3
cust_id            
c1       30  10   0
c2       40  15  30
c3        0   0  10 

prod_cd  p1  p2  p3
cust_id            
c1       30  10   0
c2       40  15  30
c3        0   0  10 

prod_cd        p1  p2  p3
cust_id grade            
c1      A      30  10   0
c2      A      40  15  30
c3      B       0   0  10 

grade       A                B           
prod_cd    p1    p2    p3   p1   p2    p3
cust_id                                  
c1       30.0  10.0   0.0  NaN  NaN   NaN
c2       40.0  15.0  30.0  NaN  NaN   NaN
c3        NaN   NaN   NaN  0.0  0.0  10.0 

prod_cd  p1  p2  p3
grade              
A        70  25  30
B         0   0  1

### stack, unstack
- `pd.DataFrame.stack()`
- `pd.DataFrame.unstack()`

In [27]:
import numpy as np
import pandas as pd
mul_index = pd.MultiIndex.from_tuples(
    [('cust_1', '2015'), ('cust_1', '2016'),
     ('cust_2', '2015'), ('cust_2', '2016')])
print(mul_index, "\n")

data = pd.DataFrame(
        data    = np.arange(16).reshape(4, 4),
        index   = mul_index,
        columns = ['prd_1', 'prd_2', 'prd_3', 'prd_4'],
        dtype   = 'int')

print(data, '\n')

# data.stack()
# 컬럼의 level이 1개 밖에 없으므로, stack(level = -1)을 별도로 명기하지 않아도 됨
data_stacked = data.stack()
print(data.stack(), '\n')
print(data.stack().index, '\n')

# indexing
print(data.stack()['cust_2']['2015'][['prd_1', 'prd_2']], '\n')

# 결측치가 있는 데이터셋을 stack() 할 때 결측값을 제거할지(dropna = True),
# 아니면 결측값을 NaN으로 유지할지(dropna = False) 설정할 수 있음

data.stack(dropna = True)
data.stack(dropna = False)

# unstack()
# unstack(level = -1), unstack(level = 0), unstack(level = 1)별로
# 어떤 level이 컬럼으로 이동해서 unstack() 되는지 보기
data_stacked_unstacked = data_stacked.unstack(level = -1)
print(data_stacked.unstack(level = -1), '\n')
print(data_stacked.unstack(level = 0), '\n')
print(data_stacked.unstack(level = 1), '\n')

# 만약 index를 dataFrame의 컬럼으로 변경하고 싶은 경우, reset_index() 사용
data_stacked_unstacked_df = data_stacked_unstacked.reset_index()
data_stacked_unstacked_df.rename(columns = {'level_0':'custID',
                                            'level_1':'year'}, inplace = True)
print(data_stacked_unstacked_df, '\n')

MultiIndex([('cust_1', '2015'),
            ('cust_1', '2016'),
            ('cust_2', '2015'),
            ('cust_2', '2016')],
           ) 

             prd_1  prd_2  prd_3  prd_4
cust_1 2015      0      1      2      3
       2016      4      5      6      7
cust_2 2015      8      9     10     11
       2016     12     13     14     15 

cust_1  2015  prd_1     0
              prd_2     1
              prd_3     2
              prd_4     3
        2016  prd_1     4
              prd_2     5
              prd_3     6
              prd_4     7
cust_2  2015  prd_1     8
              prd_2     9
              prd_3    10
              prd_4    11
        2016  prd_1    12
              prd_2    13
              prd_3    14
              prd_4    15
dtype: int64 

MultiIndex([('cust_1', '2015', 'prd_1'),
            ('cust_1', '2015', 'prd_2'),
            ('cust_1', '2015', 'prd_3'),
            ('cust_1', '2015', 'prd_4'),
            ('cust_1', '2016', 'prd_1'),
            ('cust

### pd.melt()
- reshape package의 melt, dcast 함수를 생각하면 됨

In [30]:
import numpy as np
import pandas as pd
data = pd.DataFrame({
    'cust_ID' : ['C_001', 'C_001', 'C_002', 'C_002'],
    'prd_CD'  : ['P_001', 'P_002', 'P_001', 'P_002'],
    'pch_cnt' : [1, 2, 3, 4],
    'pch_amt' : [100, 200, 300, 400]})

# pd.melt(data, id_vars = ['id1', 'id2', ...])
data_melt = pd.melt(data, id_vars = ['cust_ID', 'prd_CD'])
print(data_melt, '\n')

# pd.melt의 variable 이름, value 이름 부여하기
# : var_name, value_name paramter 
data_melt_named = pd.melt(data, id_vars = ['cust_ID', 'prd_CD'], var_name = 'pch_CD', value_name = 'pch_name')
print(data_melt_named, '\n')

# melt vs pd.pivot_table
# melt는 ID가 컬럼으로 존재하고, pd.pivot_table은 ID가 index로 들어감


  cust_ID prd_CD variable  value
0   C_001  P_001  pch_cnt      1
1   C_001  P_002  pch_cnt      2
2   C_002  P_001  pch_cnt      3
3   C_002  P_002  pch_cnt      4
4   C_001  P_001  pch_amt    100
5   C_001  P_002  pch_amt    200
6   C_002  P_001  pch_amt    300
7   C_002  P_002  pch_amt    400 

  cust_ID prd_CD   pch_CD  pch_name
0   C_001  P_001  pch_cnt         1
1   C_001  P_002  pch_cnt         2
2   C_002  P_001  pch_cnt         3
3   C_002  P_002  pch_cnt         4
4   C_001  P_001  pch_amt       100
5   C_001  P_002  pch_amt       200
6   C_002  P_001  pch_amt       300
7   C_002  P_002  pch_amt       400 



### pd.wide_to_long()
- 컬럼 이름의 앞부분과 나머지 컬럼 이름의 뒷부분을 구분하여, 컬럼 이름의 앞부분을 컬럼 이름으로, 컬럼 이름의 나머지 뒷부분을 행(row)의 원소로 해서 세로로 길게(long~) 쌓아 줌

In [41]:
import numpy as np
import pandas as pd
from pandas import DataFrame
np.random.seed(10)
data_wide = pd.DataFrame({
    "C1prd1" : {0 : "a", 1 : "b", 2 : "c"},
    "C1prd2" : {0 : "d", 1 : "e", 2 : "f"},
    "C2prd1" : {0 : 2.5, 1 : 1.2, 2 : .7},
    "C2prd2" : {0 : 3.2, 1 : 1.3, 2 : .1},
    "value" : dict(zip(range(3), np.random.randn(3)))
    })
data_wide["seq_no"] = data_wide.index
print(data_wide, '\n')

# pd.wide_to_long(data, ['col_prefix_1', 'col_prefix_2'], 
#                 i = "idx_1", j = 'idx_2')

data_long = pd.wide_to_long(data_wide, ["C1", "C2"], 
                            i = "seq_no", j = "prd")

# 결괏값이 나오지 않음
data_long = pd.wide_to_long(data_wide, ["C1", "C2"], i="seq_no", j="prd")
print(data_long, '\n')

  C1prd1 C1prd2  C2prd1  C2prd2     value  seq_no
0      a      d     2.5     3.2  1.331587       0
1      b      e     1.2     1.3  0.715279       1
2      c      f     0.7     0.1 -1.545400       2 

Empty DataFrame
Columns: [value, C1prd2, C1prd1, C2prd1, C2prd2, C1, C2]
Index: [] 



Index(['value', 'C1prd2', 'C1prd1', 'C2prd1', 'C2prd2', 'C1', 'C2'], dtype='object')

### pd.crosstab()을 사용해 교차표 생성
- 범주형 변수로 되어있는 요인(factors)별로 교차분석(cross_tabulations)해서  
  행, 열, 요인 기준 별로 빈도를 세어 도수분포표, 교차표를 만들어줌

In [49]:
import numpy as np
import pandas as pd
data = DataFrame({
        'id': ['id1', 'id1', 'id1', 'id2', 'id2', 'id3'],
        'fac_1': ['a', 'a', 'a', 'b', 'b', 'b'],
        'fac_2': ['d', 'd', 'd', 'c', 'c', 'd']})

print(pd.crosstab(data.fac_1, data.fac_2), '\n')
print(pd.crosstab(data.id, data.fac_1))
print(pd.crosstab(data.id, data.fac_2))

print("-- multiIndex --")
# Multi-index, Multi-level로 교차표 만들기
print(pd.crosstab(data.id, [data.fac_1,data.fac_2]), '\n')
print(pd.crosstab([data.fac_1,data.fac_2], data.id), '\n')

#- 교차표의 행 이름, 열 이름 부여
#- rownames = 'xx', colnames = 'xx'

# 교차표의 행 합, 열 합 추가하기 : pd.crosstab(margins = True)
# 구성비율로 교차표 만들기 : normalize = True

fac_2  c  d
fac_1      
a      0  3
b      2  1 

fac_1  a  b
id         
id1    3  0
id2    0  2
id3    0  1
fac_2  c  d
id         
id1    0  3
id2    2  0
id3    0  1
-- multiIndex --
fac_1  a  b   
fac_2  d  c  d
id            
id1    3  0  0
id2    0  2  0
id3    0  0  1 

id           id1  id2  id3
fac_1 fac_2               
a     d        3    0    0
b     c        0    2    0
      d        0    0    1 

